In [9]:
# LIBRARIES
import math
import random
import math
import statistics
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib as plt
import scipy as sp
import scipy.sparse as sparse
import seaborn as sns
from tqdm import tqdm
import subprocess as sproc

In [10]:
#____________________________________________
# CONSTANTS

TRANSFORMATIONS = ["identity", "modularity", "Laplacian", "signless_Laplacian", "general_Zagreb", "Seidel", "sum_connectivity", "transition_rw", "Bethe_Hessian", "FLCM"]

In [11]:
#____________________________________________
# SPECTRAL GRAPH FORGE

def SGF(G, transformation = "modularity", alpha = 0.8, normalization_type = "truncate", k = 6):

    #____________________________________________
    # INPUT MATRIX

    # Setting up basic parameters
    A = nx.adjacency_matrix(G).toarray()
    n = len(A)
    degrees = [G.degree[node] for node in G.nodes()]
    M = np.zeros((n, n))

    # identity transformation
    if transformation == "identity":
        M = A

    # modularity transformation
    elif transformation == "modularity":
        m = sum(degrees) / 2
        B = A - np.outer(degrees, degrees) / (2 * m)
        M = B

    # laplacian transformation
    elif transformation == "Laplacian":
        D = np.diag(degrees)
        L = D - A
        M = L

    # signless laplacian transformation
    elif transformation == "signless_Laplacian":
        D = np.diag(degrees)
        L_signless = D + A
        M = L_signless

    # Normalized Laplacian transformation
    elif transformation == "normalized_Laplacian": # TODO DELETE THIS THING
        D_inv_sqrt = np.diag(1.0 / np.sqrt(np.sum(A, axis=1)))
        L_norm = np.eye(A.shape[0]) - D_inv_sqrt @ A @ D_inv_sqrt
        M = L_norm

    # General Zagreb transformation
    elif transformation == "general_Zagreb":
        D = np.diag(degrees)
        GZ = pow(D,3) + A
        M = GZ

    # Seidel transformation
    elif transformation == "Seidel":
        A_complement = np.ones((n, n)) - A - np.eye(n)
        S = A - A_complement
        M = S

    # Sum connectivity transformation
    elif transformation == "sum_connectivity":
        SC = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if A[i][j] == 1:
                    SC[i][j] = A[i][j] * (1 / math.sqrt((degrees[i] * degrees[j])))
        M = SC

    # Transition Random Walk transformation
    elif transformation == "transition_rw":
        D = np.diag(degrees)
        P = np.linalg.inv(D) @ A
        M = P

    # Bethe-Hessian transformation
    elif transformation == "Bethe_Hessian":
        # Computing the non-backtracking matrix
        Gdirect = G.to_directed()
        S = np.zeros((len(Gdirect.edges),len(G.nodes)))
        T = np.zeros((len(G.nodes),len(Gdirect.edges)))
        for i,a in enumerate(Gdirect.edges):
            for j,b in enumerate(G.nodes):
                if a [ 1 ] == b:
                    S[i,j]=1
                if a [ 0 ] == b :
                    T[j,i] = 1
        tau = np.zeros((len(Gdirect.edges),len(Gdirect.edges)))
        for i,a in enumerate(Gdirect.edges):
            for j,b in enumerate(Gdirect.edges):
                if a[0]==b[1] and a[1]==b[0]:
                    tau[i][j] = 1
        B = S@T - tau
        # Computing the Bethe-Hessian matrix
        D = np.diag(degrees)
        I = np.eye(n, n)
        rho = max(abs(np.linalg.eigvals(B)))
        # rho = np.mean(degrees) # alternative way to compute rho
        r = pow(rho, 1/2)
        # r = sum(d**2 for v, d in nx.degree(G)) / sum(d for v, d in nx.degree(G)) - 1 # alternative way to compute r
        H = (pow(r,2) - 1) * I - r * A + D
        M = H

    elif transformation == "FLCM":
        s = np.sum((np.sum(A, axis=1)) ** 2)
        D = np.diag(degrees)
        I = np.eye(n, n)
        T = A - (1/3)*(np.outer(degrees, degrees) / (2*s)) + (1/3)*I
        M = T

    else:
        raise ValueError("Invalid transformation")

    #____________________________________________
    # LOW RANK ALPHA APPROXIMATION
    
    # Procedure for Bethe-Hessian transformation
    if transformation == "What I wanted from Bethe Hessian":
        # Computation of eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eigh(M)
        eigenvectors = eigenvectors.T

        # sorting the eigenvectors, eigenvalues
        paired_sorted_list = sorted(zip(eigenvalues, eigenvectors), key=lambda x: x[0])
        eigenvalues_sorted, eigenvectors_sorted = zip(*paired_sorted_list)

        # Computation of M_tilde
        M_tilde = np.zeros((eigenvectors_sorted[0].shape[0], eigenvectors_sorted[0].shape[0]))
        for i in range(math.ceil(alpha * len(eigenvalues) * 0.5)):
            contribution = eigenvalues_sorted[i] * np.outer(eigenvectors_sorted[i], eigenvectors_sorted[i])
            M_tilde += contribution
            if i == 0:
                pass
            else:
                contribution = eigenvalues_sorted[-i] * np.outer(eigenvectors_sorted[-i], eigenvectors_sorted[-i])
                M_tilde += contribution

    # Procedure for other transformations
    else:
        # Computation of eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eigh(M)
        eigenvectors = eigenvectors.T

        # sorting the eigenvectors, eigenvalues
        paired_sorted_list = sorted(zip(eigenvalues, eigenvectors), key=lambda x: abs(x[0]), reverse=True)
        eigenvalues_sorted, eigenvectors_sorted = zip(*paired_sorted_list)

        # Computation of M_tilde
        M_tilde = np.zeros((eigenvectors_sorted[0].shape[0], eigenvectors_sorted[0].shape[0]))
        for i in range(math.ceil(alpha * len(eigenvalues))):
            contribution = eigenvalues_sorted[i] * np.outer(eigenvectors_sorted[i], eigenvectors_sorted[i])
            M_tilde += contribution

    #____________________________________________
    # BACK TRANSFORMATION
        
    # Computation of A_tilde
    A_tilde = np.zeros((n, n))
    if transformation == "identity":
        A_tilde = M_tilde
    elif transformation == "modularity":
        A_tilde = M_tilde + np.outer(degrees, degrees) / (2 * m)
    elif transformation == "Laplacian":
        A_tilde = D - M_tilde
    elif transformation == "signless_Laplacian":
        A_tilde = M_tilde - D
    elif transformation == "normalized_Laplacian":
        D_sqrt = np.diag(np.sqrt(np.sum(A, axis=1)))
        A_tilde = D_sqrt @ (np.eye(A.shape[0]) - M_tilde) @ D_sqrt
    elif transformation == "general_Zagreb":
        A_tilde = M_tilde - pow(D,3)
    elif transformation == "Seidel":
        A_tilde = M_tilde + A_complement
    elif transformation == "sum_connectivity":
        for i in range(n):
            for j in range(n):
                if A[i][j] == 1:
                    A_tilde[i][j] = M_tilde[i][j] * math.sqrt((degrees[i] * degrees[j]))
    elif transformation == "transition_rw":
        A_tilde = D @ M_tilde
    elif transformation == "Bethe_Hessian":
        A_tilde = ((r**2 - 1) * I - M_tilde + D) * (1 / r)
    elif transformation == "FLCM":
        A_tilde = M_tilde + (1/3)*(np.outer(degrees, degrees) / (2*s)) - (1/3)*I

    #____________________________________________
    # NORMALIZATION

    # logistic
    A_dots = np.zeros((n, n))
    if normalization_type == "logistic":
        for i in range(n):
            for j in range(n):
                A_dots[i][j] = 1 / (1 + math.exp((0.5 - A_tilde[i][j])*k))

    # truncation
    elif normalization_type == "truncate":
        for i in range(n):
            for j in range(n):
                if A_tilde[i][j] < 0:
                    A_dots[i][j] = 0
                elif A_tilde[i][j] > 1:
                    A_dots[i][j] = 1
                else:
                    A_dots[i][j] = A_tilde[i][j]

    # scaling
    elif normalization_type == "scale":
        A_tilde_flattened = A_tilde.flatten()
        for i in range(n):
            for j in range(n):
                A_dots[i][j] = (A_tilde[i][j] - min(A_tilde_flattened)) / (max(A_tilde_flattened) - min(A_tilde_flattened))
    
    #____________________________________________
    # ADJACENCY MATRIX GENERATION
    
    # Bernoulli sampling
    A_prime = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            u = random.uniform(0, 1)
            if u < A_dots[i][j]:
                A_prime[i][j] = 1
                A_prime[j][i] = 1
            else:
                A_prime[i][j] = 0
                A_prime[j][i] = 0
            if i == j:
                A_prime[i][j] = 0

    W = nx.from_numpy_array(A_prime)
    return W

In [12]:
#____________________________________________
# EDIT DISTANCE

def edit_distance(A1,A2, k = 10):
    dist = np.abs((A1-A2)).sum() / 2
    return 1 - k / (k + dist)

In [13]:
#____________________________________________
# COMPARABILITY MEASURE COMPUTATION

# Euclidean distance from point (1,1,0)
def comparability_measure(avg_clustering, modularity_communities, edit_distance):
    return np.sqrt((avg_clustering - 1)**2 + (modularity_communities - 1)**2 + (edit_distance - 1)**2)

In [14]:
#____________________________________________
# STATISTICS COMPUTATION

def chart_data(graphs, alpha_nums, sample_size = 5):
    count = 0
    alphas = {v: {} for v in alpha_nums}
    for alpha in tqdm(list(alphas.keys())):
        transformation_dict = {}
        for transformation in TRANSFORMATIONS:
            transformation_dict[transformation] = {}
        for transformation in list(transformation_dict.keys()):
            measure_dict = {
                "avg_clustering_ratio_obs": [],
                "modularity_communities_ratio_obs": [],
                "edit_distance": [], #ith value corresponds to ith graph (average of sample_size runs)
                "comparability_measure": [] #ith value corresponds to ith graph (average of sample_size runs)
            }
            for graph in tqdm(graphs): 
                graph_avg_clusterings = []
                graph_modularities = []
                distances = []
                comparabilities = []
                for i in range(sample_size):
                    count += 1
                    w = SGF(graph, transformation, alpha)
                    avg_cls =nx.average_clustering(w)/ nx.average_clustering(graph)
                    mod = len(nx.community.greedy_modularity_communities(w)) / len(nx.community.greedy_modularity_communities(graph))
                    dist = edit_distance(nx.adjacency_matrix(graph), nx.adjacency_matrix(w))
                    comparability = comparability_measure(avg_cls, mod, dist)
                    graph_avg_clusterings.append(avg_cls)
                    graph_modularities.append(mod)
                    distances.append(dist)
                    comparabilities.append(comparability)
                measure_dict["avg_clustering_ratio_obs"].extend(graph_avg_clusterings)
                measure_dict["modularity_communities_ratio_obs"].extend(graph_modularities)
                avg_distances = np.mean(distances)
                avg_comparabilities = np.mean(comparabilities)
                measure_dict["edit_distance"].append(avg_distances)
                measure_dict["comparability_measure"].append(avg_comparabilities)
                
            transformation_dict[transformation] = measure_dict
            
        alphas[alpha] = transformation_dict
        
    return alphas, count

#____________________________________________
# BOXPLOTS OF AVG CLUSTERING RATIO AND MODULARITY COMMUNITIES RATIO

def boxplots(chart_data):
    for metric in ["avg_clustering_ratio_obs", "modularity_communities_ratio_obs"]:
        for alpha in list(chart_data.keys()):
            # df = pd.DataFrame(alphas[alpha])
            df = pd.DataFrame(pd.DataFrame(chart_data[alpha]).T[metric].to_dict())
            plot = sns.boxplot(data = df, palette = "ch:start=.2,rot=-.3")
            plot_title = "Avg Clustering Ratio" if metric == "avg_clustering_ratio_obs" else "Modularity Communities Ratio"
            plt.pyplot.title(f"Boxplot of {plot_title} for alpha = {round(alpha,3)}")
            plt.pyplot.ylim(0, 5)
            plot.set_xticklabels(plot.get_xticklabels(), rotation=30)
            #plt.pyplot.show()
            name = f"Boxplot of {plot_title} for alpha = {round(alpha,3)}.png"
            plt.pyplot.savefig(fname = name, bbox_inches = "tight")
            plt.pyplot.close()

#____________________________________________
# DATAFRAMES CONSTRUCTOR FOR LINE PLOTS

def distance_and_comparability_tables(chart_data, transformation):
    alphas = list(chart_data.keys())
    df_dist = pd.DataFrame(columns = alphas)
    df_comp = pd.DataFrame(columns = alphas)
    for alpha in chart_data:
        distances = chart_data[alpha][transformation]["edit_distance"]
        comparabilities = chart_data[alpha][transformation]["comparability_measure"]
        df_dist[alpha] = distances
        df_comp[alpha] = comparabilities

    return df_dist.T, df_comp.T


In [ ]:
#____________________________________________
# STATISTICS FUNCTION CALL

# # loading the facebook dataset
# df = pd.read_csv("facebookedgelist.csv", delimiter=";")
# df.columns = ["source", "target"]
# facebook = nx.from_pandas_edgelist(df, "source", "target")

# # loading the movies dataset
# df = pd.read_csv("movies_edges.csv")
# df.columns = ["source", "target", " ", " ", " "]
# movies = nx.from_pandas_edgelist(df, "source", "target")

# generating Lancichinetti graph
n = 250
tau1 = 3
tau2 = 1.5
mu = 0.1
Lancichinetti = nx.generators.community.LFR_benchmark_graph (n, tau1, tau2, mu, average_degree=2, min_community=17, seed=10)

# generating gaussian random partition graph
n = 100
s = 10
v = 10
p_in = 0.4
p_out = 0.1
GRP = nx.generators.community.gaussian_random_partition_graph(n, s, v, p_in, p_out, seed = 42)

# generating stochastic block model graph
sizes = [75, 75, 125]
probs = [[0.25, 0.05, 0.02], [0.05, 0.35, 0.07], [0.02, 0.07, 0.40]]
SBM = nx.stochastic_block_model(sizes, probs, seed=0)

# generating a windmill graph
wg = nx.windmill_graph(7, 10)

graphs = [
    Lancichinetti,
    GRP,
    SBM,
    wg
    ]
alpha_nums = np.linspace(0.1, 1, num=10)
c_data, count = chart_data(graphs, alpha_nums)

In [ ]:
#____________________________________________
# BOXPLOTS PRINT FUNCTION CALL

sns.dark_palette("#69d", reverse=True, as_cmap=True)
boxplots(c_data)

In [28]:
#____________________________________________
# PRINT OF EDIT DISTANCE AND COMPARABILITY MEASURE

for transformation in TRANSFORMATIONS:
    df_dist, df_comp = distance_and_comparability_tables(c_data, transformation)
    df_dist["avg"] = df_dist.mean(axis=1)
    # plt.pyplot.plot(df_dist)
    plt.pyplot.plot(df_dist["avg"])
    plt.pyplot.title(f"Edit Distance: {transformation}")
    name = f"Edit Distance {transformation}.png"
    plt.pyplot.savefig(fname = name, bbox_inches = "tight")
    plt.pyplot.close()

    df_comp["avg"] = df_comp.mean(axis=1)
    # plt.pyplot.plot(df_comp)
    plt.pyplot.plot(df_comp["avg"])
    plt.pyplot.title(f"Comparability measure: {transformation}")
    name = f"Comparability measure {transformation}.png"
    plt.pyplot.savefig(fname = name, bbox_inches = "tight")
    plt.pyplot.close()